conda install conda-forge::pymcr

In [ ]:
import numpy
import pandas
from matplotlib import pyplot
from scipy.signal import savgol_filter
filename = "dataframe.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0].values
xmax, xmin = 3700, 3000
data = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
data = pandas.DataFrame(savgol_filter(data.values, 7, 2, 0), index=data.index, columns=data.columns)
data.mask(data < 0, 0, inplace=True)
data.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()

In [ ]:
# initial guess (dimensionality reduction)
n = 3  # components
i = 3  # 1:SVD, 2:PCA, 3:NMF
from sklearn import decomposition
if i == 1: model = decomposition.TruncatedSVD(n_components=n).fit(data)
if i == 2: model = decomposition.PCA(n_components=n).fit(data)
if i == 3: model = decomposition.NMF(n_components=n, init='random', max_iter=200).fit(data)
init_spec = pandas.DataFrame(model.components_, columns=data.columns)
init_spec.T.plot()
pyplot.gca().invert_xaxis()
init_conc = pandas.DataFrame(model.fit_transform(data), index=data.index)
init_conc.plot()
pyplot.show()

In [ ]:
# initial guess (subtraction spectra)
y1 = data.values[1] - data.values[0]
y2 = data.values[2] - data.values[1]
y3 = data.values[-1] - data.values[-2]
init_spec = pandas.DataFrame([y1, y2, y3], columns=data.columns)
init_spec=(init_spec.T/init_spec.T.sum()).T
init_spec.T.plot()
pyplot.gca().invert_xaxis()
#init_spec.T.to_csv(filename[:len(filename)-4]+'_init_spec.csv')
pyplot.show()

In [ ]:
# initial guess (file load)
filename = "dataframe_init_spec.csv"
init_spec = pandas.read_csv(filename, header=0, index_col=0).T
init_spec.T.plot()
pyplot.gca().invert_xaxis()
pyplot.show()

In [ ]:
# MCR
from pymcr.mcr import McrAR
i = 2  # 1:OLS, 2:NNLS, 3:mix
if i == 1: mcr = McrAR(c_regr="OLS", st_regr="OLS")
if i == 2: mcr = McrAR(c_regr="NNLS", st_regr="NNLS")
if i == 3: mcr = McrAR(c_regr="OLS", st_regr="NNLS")
i = 1  # 1:spec, 2:conc, 3:random
if i == 1: mcr.fit(data.values, ST=init_spec)
if i == 2: mcr.fit(data.values, C=init_conc)
if i == 3: mcr.fit(data.values, C=numpy.random.rand(len(data.index), n))
ST_opt = pandas.DataFrame(mcr.ST_opt_.T, index=data.columns.T)
ST_opt.plot()
pyplot.gca().invert_xaxis()
C_opt = pandas.DataFrame(mcr.C_opt_, index=data.index)
C_opt.plot()
pyplot.show()

In [ ]:
# save
#ST_opt.to_csv(filename[:-4] + "_opt_spec.csv")
#C_opt.to_csv(filename[:-4] + "_opt_conc.csv")
#data.T.to_csv(filename[:-4] + "_spec.csv")